# Jieba

In [ ]:
# devide token by seperators
total_entity_list = []
sentence_entity_list = []
total_label_list = []
sentence_label_list = []

for index, entityElement in enumerate(dataToken.iloc[:, 0]):
    if entityElement not in ["，", "。", "！", "!", "？", "?"]:
        sentence_entity_list.append(entityElement)
        sentence_label_list.append(dataToken.iloc[index, 1])
    else:
        total_entity_list.append(sentence_entity_list)
        total_label_list.append(sentence_label_list)
        sentence_entity_list = []
        sentence_label_list = []
    
print("list length: ", len(total_entity_list), len(total_label_list))


# Clean up tokens and eliminate unneccesary tokens
for sentenceIndex, sentences in enumerate(total_entity_list):
    indexPunct_0 = 0
    indexPunct_1 = 0
    for tokenIndex, tokens in enumerate(sentences):
        # set up element indexes that are between two punctuation
        if total_label_list[sentenceIndex][tokenIndex] in ["x"]:
            if indexPunct_0 == 0:
                indexPunct_0 = tokenIndex
            else:
                indexPunct_1 = tokenIndex
    # remove tokens that are between two punctuations
    if indexPunct_1 == 0:
        total_entity_list[sentenceIndex][indexPunct_1] = ""
        total_label_list[sentenceIndex][indexPunct_1] = ""
    else:
        for removeIndex in range(indexPunct_0, indexPunct_1 + 1):
            total_entity_list[sentenceIndex][removeIndex] = ""
            total_label_list[sentenceIndex][removeIndex] = ""
            
    total_entity_list[sentenceIndex] = list(filter(("").__ne__, total_entity_list[sentenceIndex]))
    total_label_list[sentenceIndex]  = list(filter(("").__ne__, total_label_list[sentenceIndex]))
                
print(total_entity_list[0] , total_label_list[0])
print("list length: ", len(total_entity_list), len(total_label_list))

In [ ]:
# get entity pairs and relations

relation_pos_possible_list = ["v"]

entity_pos_possible_list = ["nr", "n"]


# get relations
all_relations_list = []
all_relations_index = []
for sentenceIndex, sentences in enumerate(total_entity_list):
    relation_list = []
    relation_index_list = []
    for tokenIndex, token in enumerate(sentences):
        if len(sentences) > 1: 
            if total_label_list[sentenceIndex][tokenIndex] in relation_pos_possible_list:
                relation_list.append(token)
                relation_index_list.append(tokenIndex)
    all_relations_list.append(relation_list)
    all_relations_index.append(relation_index_list)


# get entities
all_reformatted_entities = []
all_reformatted_index_list = []
for sentenceIndex, sentences in enumerate(total_entity_list):
    entity_index_list = []
    possible_entities = []
    reformatted_entities = []
    reformatted_index_list = []
    for tokenIndex, token in enumerate(sentences):
        if len(sentences) > 1:
            if total_label_list[sentenceIndex][tokenIndex] in entity_pos_possible_list:
                entity_index_list.append(tokenIndex)
                possible_entities.append(token)
    # combine token if situate next to each other
#     print(entity_index_list, possible_entities)
    if len(possible_entities) > 0:
        combine_entity_name = possible_entities[0]
        for possibleIndex, possibleElement in enumerate(entity_index_list):
#             print(possible_entities)
            
            isContinuous = False
            if possibleIndex != 0:
                if possibleElement == entity_index_list[possibleIndex - 1] + 1:
                    isContinuous = True
                    combine_entity_name += possible_entities[possibleIndex]
                else:
                    isContinuous = False
            
                if isContinuous == False:
                    reformatted_entities.append(combine_entity_name)
                    reformatted_index_list.append(entity_index_list[possibleIndex-1])
                    combine_entity_name = possible_entities[possibleIndex]
                    
            if possibleIndex == (len(entity_index_list) - 1):
                reformatted_entities.append(combine_entity_name)
                reformatted_index_list.append(possibleElement)
                
#             print(combine_entity_name)     
#     print(reformatted_entities)
    all_reformatted_entities.append(reformatted_entities)
    all_reformatted_index_list.append(reformatted_index_list)
                
        
# print(all_relations_list, len(all_relations_list), "\n")
# print(all_reformatted_entities, len(all_reformatted_entities), )
print(len(all_relations_list), len(all_relations_index), len(all_reformatted_entities), len(all_reformatted_index_list))

# print out parsing result
for index, element in enumerate(all_reformatted_entities):
    print(element, all_reformatted_index_list[index], all_relations_list[index], all_relations_index[index])